In [1]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
import warnings
from simul_file import mmn_queueing_jsq, queue_length_time_graph, \
                        mmn_queueing_redundancy_dos, mmn_queueing_redundancy_doc, \
                        mmn_queueing_redundancy_dos_identical, \
                        mmn_queueing_redundancy_doc_identical, \
                        mmn_queueing_redundancy_dos_weibull, \
                        mmn_queueing_redundancy_doc_weibull

In [2]:
mmn_queueing_redundancy_dos_weibull(5,5,20,3,3,simulation_time=60)

(           Time  Next Customer  Finish Time  Server1  Server1 Available Time  \
 1      0.000000       0.068856         60.0      0.0                     NaN   
 2      0.068856       0.122465         60.0      0.0                     NaN   
 3      0.122465       0.198055         60.0      0.0                     NaN   
 4      0.198055       0.227312         60.0      1.0                3.361596   
 5      0.227312       0.250511         60.0      1.0                3.361596   
 ...         ...            ...          ...      ...                     ...   
 1270  59.931631      59.931922         60.0      1.0               62.555116   
 1271  59.931922      59.974087         60.0      1.0               62.555116   
 1272  59.974087      59.979862         60.0      1.0               62.555116   
 1273  59.979862      60.072820         60.0      1.0               62.555116   
 1274  60.000000      60.072820         60.0      1.0               62.555116   
 
       Server1 Queue  Serv